In [12]:
# Copyright (C) 2018-2022 The python-bitcoin-utils developers
#
# This file is part of python-bitcoin-utils
#
# It is subject to the license terms in the LICENSE file found in the top-level
# directory of this distribution.
#
# No part of python-bitcoin-utils, including this file, may be copied,
# modified, propagated, or distributed except according to the terms contained
# in the LICENSE file.


from bitcoinutils.setup import setup
from bitcoinutils.utils import to_satoshis
from bitcoinutils.transactions import Transaction, TxInput, TxOutput
from bitcoinutils.keys import P2pkhAddress, P2shAddress, PrivateKey
from bitcoinutils.script import Script


# always remember to setup the network
setup('testnet')

'testnet'

In [13]:
#
# This script creates a P2SH address containing a P2PK script and sends
# some funds to it
#

# create transaction input from tx id of UTXO (contained 0.1 tBTC)
txin = TxInput('76464c2b9e2af4d63ef38a77964b3b77e629dddefc5cb9eb1a3645b1608b790f', 0)

In [14]:
# address we are spending from
from_addr = P2pkhAddress('n4bkvTyU1dVdzsrhWBqBw8fEMbHjJvtmJR')
# secret key of address that we are trying to spent
sk = PrivateKey('cTALNpTpRbbxTCJ2A5Vq88UxT44w1PE2cYqiB3n4hRvzyCev1Wwo')


In [15]:
# create transaction output using P2SH scriptPubKey (locking script)
# (the recipient will give us the final address  but for now we create it
# for demonstration purposes)
#

# secret key corresponding to the pubkey needed for the P2SH (P2PK) transaction
p2pk_sk = PrivateKey('cRvyLwCPLU88jsyj94L7iJjQX5C2f8koG4G2gevN4BeSGcEvfKe9')
p2pk_pk = p2pk_sk.get_public_key().to_hex()
redeem_script = Script([p2pk_pk, 'OP_CHECKSIG'])
txout = TxOutput(to_satoshis(0.09), redeem_script.to_p2sh_script_pub_key() )
# no change address - the remaining 0.01 tBTC will go to miners)
txout

{'amount': 9000000, 'script_pubkey': ['OP_HASH160', '2910fc0b1b7ab6c9789c5a67c22c5bcde5b90390', 'OP_EQUAL']}

In [16]:
# create transaction from inputs/outputs -- default locktime is used
tx = Transaction([txin], [txout])

# print raw transaction
print("\nRaw unsigned transaction:\n" + tx.serialize())


Raw unsigned transaction:
02000000010f798b60b145361aebb95cfcdedd29e6773b4b96778af33ed6f42a9e2b4c46760000000000ffffffff01405489000000000017a9142910fc0b1b7ab6c9789c5a67c22c5bcde5b903908700000000


In [17]:
# use the private key corresponding to the address that contains the
# UTXO we are trying to spend to create the signature for the txin
sig = sk.sign_input(tx, 0, from_addr.to_script_pub_key() )
sig

'304402206f4027d0a1720ea4cc68e1aa3cc2e0ca5996806971c0cd7d40d3aa4309d4761802206c5d9c0c26dec8edab91c1c3d64e46e4dd80d8da1787a9965ade2299b41c380301'

In [18]:
# get public key as hex
pk = sk.get_public_key()
pk = pk.to_hex()
pk

'02d82c9860e36f15d7b72aa59e29347f951277c21cd4d34822acdeeadbcff8a546'

In [19]:
# set the scriptSig (unlocking script)
txin.script_sig = Script([sig, pk])
signed_tx = tx.serialize()

# print raw signed transaction ready to be broadcasted
print("\nRaw signed transaction:\n" + signed_tx)
print("\nTxId:", tx.get_txid())


Raw signed transaction:
02000000010f798b60b145361aebb95cfcdedd29e6773b4b96778af33ed6f42a9e2b4c4676000000006a47304402206f4027d0a1720ea4cc68e1aa3cc2e0ca5996806971c0cd7d40d3aa4309d4761802206c5d9c0c26dec8edab91c1c3d64e46e4dd80d8da1787a9965ade2299b41c3803012102d82c9860e36f15d7b72aa59e29347f951277c21cd4d34822acdeeadbcff8a546ffffffff01405489000000000017a9142910fc0b1b7ab6c9789c5a67c22c5bcde5b903908700000000

TxId: 61a6765f7c43a5e74aa733d6210b9b696cfd96d6863e438e7e0d5380d5d2d9e0
